In [2]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import sys
sys.path.append(f'/home/beans/bespoke')

from models import EffNet
from constants import *
from imports import *
from rw_dataloader import RealWorldDataloader
from blender_dataloader import BlenderDataloader
from train_utils import *

torch.__version__, torch.cuda.device_count(), torch.cuda.get_device_name(torch.cuda.current_device())

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


('1.10.0', 2, 'NVIDIA GeForce RTX 3090')

In [2]:
import wandb
run_id = wandb.util.generate_id(); run_id

'1spp96yb'

In [3]:
wandb.init(id='mv5z50t6', project="carla", resume="allow")

wandb: Currently logged in as: rgilman33 (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.13.7 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


In [4]:
m = EffNet().to(device) # 13M params, 11.6M without RNN, 
sum([torch.numel(p) for p in m.parameters()]) / 1000

18303.919

In [5]:
m.load_state_dict(torch.load(f"{BESPOKE_ROOT}/models/m.torch"), strict=False)

<All keys matched successfully>

In [6]:
scaler = torch.cuda.amp.GradScaler() 
opt = torch.optim.Adam(m.parameters(), lr=5e-5)

In [7]:
%%time
BS = 32
dataloader_trn = BlenderDataloader(path_stem="trn", bs=BS)

CPU times: user 5min 27s, sys: 1min 57s, total: 7min 25s
Wall time: 2min 24s


In [8]:
%%time
#rw_dataloader = RealWorldDataloader()

CPU times: user 4 µs, sys: 1 µs, total: 5 µs
Wall time: 9.06 µs


In [9]:
set_lr(3e-4)

In [10]:
model_stem = "12.15"
START_E = 16

In [11]:
n_epochs = 10000
log_wandb = True

for e in range(START_E, n_epochs):
    
    lr = get_lr()
    opt.param_groups[0]['lr'] = lr if lr!=42 else (6e-4 if e < 35 else 3e-4)

    # Train
    print(f"EPOCH {e}\n")
    run_epoch(dataloader_trn, m, opt=opt, scaler=scaler, 
              train=True, backwards=True, log_wandb=log_wandb, wandb=wandb, updates_per_epoch=2560)
    
    print("\n\n Eval...")
    
    #eval_rw(rw_dataloader, m, wandb) # will log directly to wandb
        
    # Saving model checkpoint each epoch
    torch.save(m.state_dict(), f"{BESPOKE_ROOT}/models/m_{model_stem}_e{e}.torch")
    

EPOCH 16



/home/beans/bespoke/constants.py:249: RuntimeWarning: Mean of empty slice
  for k in r: r[k] = np.round(np.nanmean(np.array(r[k])), 8)




 {'lat_losses/trn_control_loss': 0.00486821, 'lat_losses/trn_headings_loss': 0.00740483, 'lat_losses/trn_curvatures_loss': 0.05215955, 'lat_losses/trn_rolls_loss': 0.72779846, 'lat_losses/trn_zs_loss': 0.08746886, 'lon_losses/trn_approaching_stop_loss': 0.01118259, 'lon_losses/trn_stop_dist_loss': 0.00581249, 'lon_losses/trn_has_lead_loss': 0.01537094, 'lon_losses/trn_lead_dist_loss': 0.00734505, 'lon_losses/trn_lead_speed_loss': 0.28638148, 'consistency losses/trn_steer_cost': nan, 'aux losses/trn_unc_loss': 1.16491318, 'aux losses/trn_pitch_loss': 1.38430786, 'aux losses/trn_yaw_loss': 1.31655121, 'consistency losses/trn_%_updates_w_torque_delta_loss': 0.0, 'consistency losses/trn_torque_loss': 2707.3702631, 'timing/get batch from dataloader': 0.00451249, 'timing/model forward': 0.05345956, 'timing/calc losses': 0.16462006, 'timing/backwards': 0.93931082, 'timing/get worst': 0.16169794, 'timing/logging': 5.16e-06, 'timing/trn update': 1.32360937, 'logistical/obs_consumed_per_second

wandb: Network error (ConnectionError), entering retry loop.




 {'timing/get batch from dataloader': 0.00548927, 'timing/model forward': 0.04495823, 'timing/calc losses': 0.16394969, 'timing/backwards': 0.93920039, 'timing/get worst': 0.16669009, 'timing/logging': 0.00862501, 'timing/trn update': 1.32891631, 'logistical/obs_consumed_per_second': 24.2421875, 'logistical/obs_generated_per_second': 84.0078125, 'logistical/slowest_runner_obs_per_sec': 6.0, 'logistical/data_consumption_ratio': 0.28872009, 'logistical/manual_train_pause': 0.0, 'lat_losses/trn_control_loss': 0.00428359, 'lat_losses/trn_headings_loss': 0.00658406, 'lat_losses/trn_curvatures_loss': 0.05311966, 'lat_losses/trn_rolls_loss': 0.15082765, 'lat_losses/trn_zs_loss': 0.16994476, 'lon_losses/trn_approaching_stop_loss': 0.01043377, 'lon_losses/trn_stop_dist_loss': 0.00460411, 'lon_losses/trn_has_lead_loss': 0.01199806, 'lon_losses/trn_lead_dist_loss': 0.00395078, 'lon_losses/trn_lead_speed_loss': 0.19718516, 'consistency losses/trn_steer_cost': nan, 'aux losses/trn_unc_loss': 1.15

...



 {'timing/get batch from dataloader': 0.00625214, 'timing/model forward': 0.04525295, 'timing/calc losses': 0.16378636, 'timing/backwards': 0.95356678, 'timing/get worst': 0.17197902, 'timing/logging': 0.008299, 'timing/trn update': 1.34914042, 'logistical/obs_consumed_per_second': 23.9296875, 'logistical/obs_generated_per_second': 88.9296875, 'logistical/slowest_runner_obs_per_sec': 6.0, 'logistical/data_consumption_ratio': 0.26899973, 'logistical/manual_train_pause': 0.0, 'lat_losses/trn_control_loss': 0.00266661, 'lat_losses/trn_headings_loss': 0.002576, 'lat_losses/trn_curvatures_loss': 0.01573914, 'lat_losses/trn_rolls_loss': 0.28002453, 'lat_losses/trn_zs_loss': 0.19648075, 'lon_losses/trn_approaching_stop_loss': 0.01317052, 'lon_losses/trn_stop_dist_loss': 0.0026341, 'lon_losses/trn_has_lead_loss': 0.00098801, 'lon_losses/trn_lead_dist_loss': 0.00622368, 'lon_losses/trn_lead_speed_loss': 0.41738915, 'consistency losses/trn_steer_cost': nan, 'aux losses/trn_unc_loss': 1.2994155



 {'timing/get batch from dataloader': 0.00369619, 'timing/model forward': 0.04492982, 'timing/calc losses': 0.16371663, 'timing/backwards': 0.95553705, 'timing/get worst': 0.16301672, 'timing/logging': 0.0098456, 'timing/trn update': 1.34074568, 'logistical/obs_consumed_per_second': 24.1484375, 'logistical/obs_generated_per_second': 80.6015625, 'logistical/slowest_runner_obs_per_sec': 5.8671875, 'logistical/data_consumption_ratio': 0.29984532, 'logistical/manual_train_pause': 0.0, 'lat_losses/trn_control_loss': 0.00383109, 'lat_losses/trn_headings_loss': 0.00426354, 'lat_losses/trn_curvatures_loss': 0.02823102, 'lat_losses/trn_rolls_loss': 0.20632982, 'lat_losses/trn_zs_loss': 0.15402174, 'lon_losses/trn_approaching_stop_loss': 0.00858374, 'lon_losses/trn_stop_dist_loss': 0.0032742, 'lon_losses/trn_has_lead_loss': 0.01601553, 'lon_losses/trn_lead_dist_loss': 0.00544257, 'lon_losses/trn_lead_speed_loss': 0.18589449, 'consistency losses/trn_steer_cost': nan, 'aux losses/trn_unc_loss': 



 {'timing/get batch from dataloader': 0.00386294, 'timing/model forward': 0.04496728, 'timing/calc losses': 0.16397428, 'timing/backwards': 0.93960655, 'timing/get worst': 0.16148724, 'timing/logging': 0.00886592, 'timing/trn update': 1.32276818, 'logistical/obs_consumed_per_second': 24.390625, 'logistical/obs_generated_per_second': 83.8515625, 'logistical/slowest_runner_obs_per_sec': 5.0625, 'logistical/data_consumption_ratio': 0.28977916, 'logistical/manual_train_pause': 0.0, 'lat_losses/trn_control_loss': 0.00410231, 'lat_losses/trn_headings_loss': 0.00386563, 'lat_losses/trn_curvatures_loss': 0.02371418, 'lat_losses/trn_rolls_loss': 0.23701, 'lat_losses/trn_zs_loss': 0.19755268, 'lon_losses/trn_approaching_stop_loss': 0.01195434, 'lon_losses/trn_stop_dist_loss': 0.00156975, 'lon_losses/trn_has_lead_loss': 0.03448393, 'lon_losses/trn_lead_dist_loss': 0.0094995, 'lon_losses/trn_lead_speed_loss': 0.23591602, 'consistency losses/trn_steer_cost': nan, 'aux losses/trn_unc_loss': 1.3225



 {'timing/get batch from dataloader': 0.00448908, 'timing/model forward': 0.04502369, 'timing/calc losses': 0.16373774, 'timing/backwards': 0.94358665, 'timing/get worst': 0.17083726, 'timing/logging': 0.01066754, 'timing/trn update': 1.33834635, 'logistical/obs_consumed_per_second': 24.0546875, 'logistical/obs_generated_per_second': 86.6796875, 'logistical/slowest_runner_obs_per_sec': 6.0390625, 'logistical/data_consumption_ratio': 0.27725456, 'logistical/manual_train_pause': 0.0, 'lat_losses/trn_control_loss': 0.00527854, 'lat_losses/trn_headings_loss': 0.00482494, 'lat_losses/trn_curvatures_loss': 0.04437685, 'lat_losses/trn_rolls_loss': 0.38227749, 'lat_losses/trn_zs_loss': 0.11832619, 'lon_losses/trn_approaching_stop_loss': 0.01112312, 'lon_losses/trn_stop_dist_loss': 0.00299316, 'lon_losses/trn_has_lead_loss': 0.00166587, 'lon_losses/trn_lead_dist_loss': 0.001997, 'lon_losses/trn_lead_speed_loss': 0.11212438, 'consistency losses/trn_steer_cost': nan, 'aux losses/trn_unc_loss': 

KeyboardInterrupt: 

In [12]:
torch.save(m.state_dict(), f"{BESPOKE_ROOT}/models/m_{model_stem}_e55.torch")

In [13]:
torch.save(m.state_dict(), f"{BESPOKE_ROOT}/models/m.torch")

In [14]:
set_should_stop(True)

In [10]:
%%time

eval_rw(rw_dataloader, m, wandb) # will log directly to wandb

/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/torch/utils/checkpoint.py:25: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")


CPU times: user 14.3 s, sys: 417 ms, total: 14.7 s
Wall time: 14 s
